<a href="https://colab.research.google.com/github/scottthomaswx/taf-verification/blob/main/TAF_Verification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load Needed Packages

In [1]:
# @title
%%capture
!pip install datetime
!pip install pytz
!pip install pandas
!pip install matplotlib
!pip install numpy
import pytz
import os
import sys
import requests
from datetime import timedelta
import datetime as datetime
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
!rm -rf sample_data/

#Enter TAF Start Time (UTC) and Airport ID

In [2]:
year = 'YYYY' #@param {type:"string"}
month = 'MM' #@param {type:"string"}
day = 'DD' #@param {type:"string"}
hour = '00' #@param ["00", "06", "12", "18"] {type:"string"}
airport = 'KXYZ' #@param {type:"string"}

timezone = 'UTC'
tz = pytz.timezone(timezone)
start = tz.localize(datetime(int(year),int(month),int(day),int(hour)))

Extended_Taf_List = ['KAUS', 'KATL', 'KBDL', 'KBOS', 'KBWI', 'KBZN', 'KCLE', 'KCLT', 'KCVG', 'KDCA', 'KDEN', 'KDFW', 'KDTW', 'KEWR', 'KFLL', 'KIAD', 'KIAH', 'KIND',
                     'KJFK', 'KLAS', 'KLAX', 'KLGA', 'KMCO', 'KMDW', 'KMEM', 'KMIA', 'KMKE', 'KMSP', 'KMSY', 'KOAK', 'KONT', 'KORD', 'KPHL', 'KPHX', 'KPIT', 'KSAN',
                     'KSDF', 'KSEA', 'KSFO', 'KSLC', 'KSTL', 'KSWF', 'KSAT', 'KTPA', 'KTEB', 'PANC', 'PAFA', 'PGUM', 'PHNL']

if airport in Extended_Taf_List:
  end_time = start + timedelta(hours=30)
  end_year = end_time.strftime('%Y')
  end_month = end_time.strftime('%m')
  end_day = end_time.strftime('%d')
  end_hour = end_time.strftime('%H')
else:
  end_time = start + timedelta(hours=24)
  end_year = end_time.strftime('%Y')
  end_month = end_time.strftime('%m')
  end_day = end_time.strftime('%d')
  end_hour = end_time.strftime('%H')

if hour == '00':
    start_time = start - timedelta(hours=1)
    start_hour = start_time.strftime('%H')
    start_day = start_time.strftime('%d')
    start_month = start_time.strftime('%m')
    start_year = start_time.strftime('%Y')

else:
  start_day = start.strftime('%d')
  start_time = start - timedelta(hours=1)
  start_hour = start_time.strftime('%H')
  start_month = start_time.strftime('%m')
  start_year = start_time.strftime('%Y')

old_TAF_Date = datetime.now(pytz.utc) - timedelta(days = 14)
if start < old_TAF_Date:
  useArchive = 'True'
else:
  useArchive = 'False'

#Enter Synoptic API Token (Required)

In [3]:
API_Token = '' #@param {type:"string"}
API_ROOT = "https://api.synopticdata.com/v2/"

#Get Observations For TAF Period

In [4]:
# @title
def mesowest_get_timeseries(api_args):
    station = api_args["stid"]
    dst_file = f'{station}.txt'
    with open(dst_file,'w') as fout:
        header = f'date_time,station,visby,cigs\n'
        fout.write(header)
        api_request_url = os.path.join(API_ROOT, "stations/timeseries")
        req = requests.get(api_request_url, params=api_args)
        jas_ts = req.json()
        for s in range(0,len(jas_ts['STATION'])):
            try:
                station = jas_ts['STATION'][s]
                stn_id = station['STID']
                ob_times = station['OBSERVATIONS']['date_time']
                visby = station['OBSERVATIONS']['visibility_set_1']
                cigs = station['OBSERVATIONS']['ceiling_set_1']

                for dt in range(0,len(ob_times)):
                            if ob_times[dt] is not None:
                              ob_time = ob_times[dt][:-1]
                              v = visby[dt]
                              c = cigs[dt]
                              line = f'{ob_time},{stn_id},{v}, {c}\n'
                              fout.write(line)
            except:
                pass
        return

api_args = {"token":API_Token,
            "stid": airport,
            "start": f"{start_year}{start_month}{start_day}{start_hour}50",
            "end": f"{end_year}{end_month}{end_day}{end_hour}00",
            "status":"active",
            "units":"temp|F",
            "hfmetars":'0'}
mesowest_get_timeseries(api_args)

df = pd.read_csv(f'{airport}.txt', sep=",")
df = df.drop(df.index[-1])

for r in range(len(df['cigs'])):
  if df.cigs[r] != ' None':
    cigft = float(df.cigs[r]) * 3.281
    df.loc[r, "cigs_ft"] = round(cigft, -2)


for i in range(len(df['cigs_ft'])):
  if np.isnan(df['cigs_ft'][i]):
    df.loc[i, "adj_cigs"] = 7000
  elif float(df.cigs_ft[i]) >= 7000.0:
    df.loc[i, "adj_cigs"] = 7000
  else:
    df.loc[i, "adj_cigs"] = df.cigs_ft[i]

for q in range(len(df['visby'])):
  if float(df.visby[q]) >= 7.0:
    df.loc[q, "adj_visby"] = 7.0
  else:
    df.loc[q, "adj_visby"] = df.visby[q]

for a in range(len(df['date_time'])):

  checktime =  datetime.strptime(df['date_time'][a], '%Y-%m-%dT%H:%M:%S')
  checktime = tz.localize(checktime)
  timediff = checktime - start
  timediff = timediff.total_seconds()
  hours = int(timediff // 3600)
  remaining_secs = timediff % 3600
  minutes = int(remaining_secs // 60) # Integer minutes
  count = hours + (minutes/60)
  df.loc[a, "hour"] = count
  df.loc[0,'hour'] = 0

#Pull and Parse TAF

In [7]:
if useArchive == 'False':
  station = airport
  tafdst_file = f'TAF_{station}.txt'
  with open(tafdst_file,'w') as fout:
          header = f'fmgpnum,from,to,type,visby,cigs\n'
          fout.write(header)
          tafdf = requests.get(f'https://aviationweather.gov/api/data/taf?ids={airport}&format=json&metar=false&time=valid&date={year}{month}{day}_{hour}0000Z')
          taf_js = tafdf.json()[0]
          filter_js = taf_js['fcsts']
          for s in range(0,len(filter_js)):
              try:
                  fm_groupnum = filter_js[s]['timeGroup']
                  visby = filter_js[s]['visib']
                  typeof = filter_js[s]['fcstChange']

                  clouds = filter_js[s]['clouds']
                  if clouds[0]['cover'] == 'BKN' or 'OVC':
                    cigs = clouds[0]['base']
                  elif clouds[0]['cover'] == 'SKC':
                    cigs = 7000
                  elif clouds[0]['cover'] == 'FEW' or 'SCT':
                      try:
                        if clouds[1]['cover'] == 'BKN' or 'OVC':
                          cigs = clouds[1]['base']
                      except:
                        cigs = 7000

                  starttime = filter_js[s]['timeFrom']
                  endtime = filter_js[s]['timeTo']

                  fn = fm_groupnum
                  s = starttime
                  e = endtime
                  v = visby
                  c = cigs
                  f = typeof
                  line = f'{fn},{s},{e},{f},{v},{c}\n'
                  fout.write(line)
              except:
                try:
                  fm_groupnum = filter_js[s]['timeGroup']
                  typeof = filter_js[s]['fcstChange']

                  clouds = filter_js[s]['clouds']
                  if clouds[0]['cover'] == 'BKN' or 'OVC':
                    cigs = clouds[0]['base']
                  elif clouds[0]['cover'] == 'SKC':
                    cigs = 7000
                  elif clouds[0]['cover'] == 'FEW' or 'SCT':
                      try:
                        if clouds[1]['cover'] == 'BKN' or 'OVC':
                          cigs = clouds[1]['base']
                      except:
                        cigs = 7000

                  starttime = filter_js[s]['timeFrom']
                  endtime = filter_js[s]['timeTo']

                  fn = fm_groupnum
                  s = starttime
                  e = endtime
                  c = cigs
                  f = typeof
                  line = f'{fn},{s},{e},{f},{c}\n'
                  fout.write(line)
                except:
                  try:
                    fm_groupnum = filter_js[s]['timeGroup']
                    visby = filter_js[s]['visib']
                    typeof = filter_js[s]['fcstChange']

                    starttime = filter_js[s]['timeFrom']
                    endtime = filter_js[s]['timeTo']

                    fn = fm_groupnum
                    s = starttime
                    e = endtime
                    v = visby
                    f = typeof
                    line = f'{fn},{s},{e},{f},{v},\n'
                    fout.write(line)
                  except:
                      pass

  # @title
  tafdf = pd.read_csv(f'TAF_{airport}.txt', sep=",")

  for f in range(len(tafdf['from'])):
    epochtimefrom = tafdf['from'][f]
    time = datetime.fromtimestamp(epochtimefrom)
    day = time.strftime('%d')
    month = time.strftime('%m')
    year = time.strftime('%Y')
    hour = time.strftime('%H')
    minute = time.strftime('%M')
    tafdf.loc[f, 'fromtime'] = f'{year}-{month}-{day}T{hour}:{minute}'

  for t in range(len(tafdf['to'])):
    epochtimefrom = tafdf['to'][t]
    time = datetime.fromtimestamp(epochtimefrom)
    day = time.strftime('%d')
    month = time.strftime('%m')
    year = time.strftime('%Y')
    hour = time.strftime('%H')
    minute = time.strftime('%M')
    tafdf.loc[t, 'totime'] = f'{year}-{month}-{day}T{hour}:{minute}'

  tafdf.loc[0, 'type'] = 'FM'

  for vis in range(len(tafdf['visby'])):
    if tafdf.visby[vis] == '6+':
      tafdf.loc[vis, 'visby'] = '7.0'
    else:
      tafdf.loc[vis, 'visby'] = float(tafdf.visby[vis])

  for cig in range(len(tafdf['cigs'])):
    if float(tafdf.cigs[cig]) > 7000:
      tafdf.loc[cig, 'cigs'] = '7000'

  for ttime in range(len(tafdf['fromtime'])):
    checktime =  datetime.strptime(tafdf['fromtime'][ttime], '%Y-%m-%dT%H:%M')
    checktime = tz.localize(checktime)
    timediff = checktime - start
    timediff = timediff.total_seconds()
    hours = int(timediff // 3600)
    remaining_secs = timediff % 3600
    minutes = int(remaining_secs // 60) # Integer minutes
    count = hours + (minutes/60)
    tafdf.loc[ttime, "starthour"] = count
    tafdf.loc[0,'starthour'] = 0

  for etime in range(len(tafdf['totime'])):
    checktime =  datetime.strptime(tafdf['totime'][etime], '%Y-%m-%dT%H:%M')
    checktime = tz.localize(checktime)
    timediff = checktime - start
    timediff = timediff.total_seconds()
    hours = int(timediff // 3600)
    remaining_secs = timediff % 3600
    minutes = int(remaining_secs // 60) # Integer minutes
    count = hours + (minutes/60)
    tafdf.loc[etime, "endhour"] = count


  fmgroup = tafdf.query('type == "FM"')
  tmpo = tafdf.query('type == "TEMPO"')
  pb30 = tafdf.query('type == "PROB"')

  fmgroup = fmgroup.reset_index(drop=True)
  tmpo = tmpo.reset_index(drop=True)
  pb30 = pb30.reset_index(drop=True)

  if airport in Extended_Taf_List:
    addlast = pd.DataFrame({'fmgpnum': [(fmgroup.fmgpnum.iloc[-1]) +1], 'from': [''], 'to': [''], 'type':['FM'], 'visby': [fmgroup.visby.iloc[-1]], 'cigs': [fmgroup.cigs.iloc[-1]], 'starthour': ['30'], 'endhour': ['30']})
  elif airport not in Extended_Taf_List:
    addlast = pd.DataFrame({'fmgpnum': [(fmgroup.fmgpnum.iloc[-1]) +1],'from': [''], 'to': [''], 'type':['FM'], 'visby': [fmgroup.visby.iloc[-1]], 'cigs': [fmgroup.cigs.iloc[-1]], 'starthour': ['24'], 'endhour': ['24']})

  fmgroup = pd.concat([fmgroup, addlast], ignore_index=True)

  if IndexError:
    warnings.warn('TAF Data Is Unavailable For This Issuance Time')

elif useArchive == 'True':
  taf_url = f'https://mesonet.agron.iastate.edu/cgi-bin/request/taf.py?station={airport}&ets={year}-{month}-{day}T{hour}:00Z&sts={start_year}-{start_month}-{start_day}T{start_hour}:00Z&fmt=csv&last=1'
  tafdf  = pd.read_csv(taf_url, sep=",")
  tafdf = tafdf

  for tafcigloop in range(len(tafdf['skyc'])):
    checkcig = tafdf.skyc[tafcigloop]
    clean_checkcig = checkcig.strip("'[]'")
    checkcig_list = clean_checkcig.split(',')

    chgt = tafdf.skyl[tafcigloop]
    clean_chgt = chgt.strip("'[]'")
    chgt_list = clean_chgt.split(',')

    if checkcig_list[0] == 'BKN' or checkcig_list[0] == 'OVC':
        tafdf.loc[tafcigloop, 'cigs'] = float(chgt_list[0])
    elif checkcig_list[0] == 'SKC':
        tafdf.loc[tafcigloop, 'cigs'] = 7000
    elif checkcig_list[0] == 'FEW' or checkcig_list[0] == 'SCT':
        try:
          if checkcig_list[1] == 'BKN' or checkcig_list[1] == 'OVC':
            tafdf.loc[tafcigloop, 'cigs'] = float(chgt_list[1])
        except:
          tafdf.loc[tafcigloop, 'cigs'] = 7000

  for tafvisloop in range(len(tafdf['visibility'])):
    checkvis = tafdf.visibility[tafvisloop]
    if checkvis > 6.0:
      tafdf.loc[tafvisloop, 'visby'] = 7.0
    else:
      tafdf.loc[tafvisloop, 'visby'] = checkvis

  tafdf['adj_cigs'] = tafdf['cigs'].copy()

  for st in range(len(tafdf['fx_valid'])):
    groupstart = tafdf.fx_valid[st]
    checktime =  datetime.strptime(groupstart, '%Y-%m-%d %H:%M')
    checktime = tz.localize(checktime)
    timediff = checktime - start
    timediff = timediff.total_seconds()
    hours = int(timediff // 3600)
    remaining_secs = timediff % 3600
    minutes = int(remaining_secs // 60) # Integer minutes
    count = hours + (minutes/60)
    tafdf.loc[st, "starthour"] = count
    tafdf.loc[0,'starthour'] = 0

  for et in range(len(tafdf['fx_valid_end'])):
    groupend = tafdf.fx_valid_end[et]
    if isinstance(groupend, str):
      checktime =  datetime.strptime(groupend, '%Y-%m-%d %H:%M')
      checktime = tz.localize(checktime)
      timediff = checktime - start
      timediff = timediff.total_seconds()
      hours = int(timediff // 3600)
      remaining_secs = timediff % 3600
      minutes = int(remaining_secs // 60) # Integer minutes
      count = hours + (minutes/60)
      tafdf.loc[et, "endhour"] = count

  fmgroup = tafdf.query('ftype == "Observation" or ftype == "Forecast"')
  fmgroup = fmgroup.reset_index(drop=True)

  tmpo = tafdf.query('ftype == "Temporary"')
  tmpo = tmpo.reset_index(drop=True)

  pb30 = tafdf.query('ftype == "Probability"')
  pb30 = pb30.reset_index(drop=True)


  if airport in Extended_Taf_List:
      addlast = pd.DataFrame({'station': [(fmgroup.station.iloc[-1])], 'valid': [''], 'raw': [''], 'is_tempo': [False], 'sknt': [''], 'drct': [''], 'visibility': [fmgroup.visby.iloc[-1]], 'skyc': [fmgroup.skyc.iloc[-1]], 'skyl': [fmgroup.skyl.iloc[-1]], 'ws_level': [''],
                              'ftype': ['Forecast'], 'cigs': [fmgroup.cigs.iloc[-1]], 'visby': [fmgroup.visby.iloc[-1]],'starthour': ['30'], 'endhour': ['30'], 'adj_cigs': [fmgroup.adj_cigs.iloc[-1]]})
  elif airport not in Extended_Taf_List:
      addlast = pd.DataFrame({'station': [(fmgroup.station.iloc[-1])], 'valid': [''], 'raw': [''], 'is_tempo': [False], 'sknt': [''], 'drct': [''], 'visibility': [fmgroup.visby.iloc[-1]], 'skyc': [fmgroup.skyc.iloc[-1]], 'skyl': [fmgroup.skyl.iloc[-1]], 'ws_level': [''],
                              'ftype': ['Forecast'], 'cigs': [fmgroup.cigs.iloc[-1]], 'visby': [fmgroup.visby.iloc[-1]],'starthour': ['24'], 'endhour': ['24'], 'adj_cigs': [fmgroup.adj_cigs.iloc[-1]]})

  fmgroup = pd.concat([fmgroup, addlast], ignore_index=True)

#Plot Data

In [ ]:
# @title
fig = plt.figure(figsize=(10, 10), facecolor='white', edgecolor="black", linewidth =6)

ax1 = plt.subplot(2,1,1)
ax1.set_title(f'{airport} Ceiling Verification for {year}-{month}-{day} {hour} UTC TAF')
ax1.axhline(y=3000, color='skyblue', linestyle='--')
ax1.axhline(y=2000, color='blue', linestyle='--')
ax1.axhline(y=1000, color='red', linestyle='--')
ax1.axhline(y=500, color='hotpink', linestyle='--')
ax1.axhline(y=200, color='purple', linestyle='--')

cigline = ax1.plot(df['hour'], df['adj_cigs'], color = 'darkorange', label = 'Observed Ceiling', linewidth = 5, zorder=  1)
fcstcig = ax1.step(fmgroup['starthour'], fmgroup['cigs'], color = 'green', label = 'Forecast Ceiling', linewidth = 5, zorder = 2)


for tm in range(len(tmpo['cigs'])):
  if not np.isnan(tmpo.cigs[tm]):
    ax1.scatter(tmpo.starthour[tm], tmpo.cigs[tm], color = 'black', marker = '^', zorder = 3, s = 100)
    ax1.scatter(tmpo.endhour[tm], tmpo.cigs[tm], color = 'black', marker = '^', zorder = 3, s = 100)
    x_coords = [tmpo.starthour[tm], tmpo.endhour[tm]]
    y_coords = [tmpo.cigs[tm], tmpo.cigs[tm]]
    ax1.plot(x_coords, y_coords, color = 'black', linewidth = 2, label = 'TEMPO', zorder = 3)
for pr in range(len(pb30['cigs'])):
  if not np.isnan(pb30.cigs[pr]):
    ax1.scatter(pb30.starthour[pr], pb30.cigs[pr], color = 'maroon', marker = '^', zorder = 3, s = 100)
    ax1.scatter(pb30.endhour[pr], pb30.cigs[pr], color = 'maroon', marker = '^', zorder = 3, s = 100)
    x_coords = [pb30.starthour[pr], pb30.endhour[pr]]
    y_coords = [pb30.cigs[pr], pb30.cigs[pr]]
    ax1.plot(x_coords, y_coords, color = 'maroon', linewidth = 2, label = 'PROB30', zorder = 3)

ax1.set_ylim(0,7500)
ax1.set_ylabel('Ceiling Height (ft)')

ax1.set_yticks([0,200, 500,1000,2000,3000,4000,5000,6000,7000])
ax1.set_yticklabels(['0','200','500', '1000', '2000', '3000', '4000', '5000', '6000', '>6000'])

ax1.set_xlabel('Hours Since TAF Began')
ax1.legend(loc = 'upper right')

if airport in Extended_Taf_List:
  ax1.set_xlim(-0.1,29.5)
  ax1.set_xticks(np.arange(0,29.5,1))
elif airport not in Extended_Taf_List:
  ax1.set_xlim(-0.1,23.5)
  ax1.set_xticks(np.arange(0,23.5,1))

ax2 = plt.subplot(2,1,2)
ax2.set_title(f'{airport} Visibility Verification {year}-{month}-{day} {hour} UTC TAF')
ax2.axhline(y=5, color='skyblue', linestyle='--')
ax2.axhline(y=3, color='red', linestyle='--')
ax2.axhline(y=1, color='hotpink', linestyle='--')
ax2.axhline(y=0.5, color='purple', linestyle='--')

visline = ax2.plot(df['hour'], df['adj_visby'], color = 'darkorange', label = 'Observed Visibility', linewidth = 5, zorder = 1)
fcstvis = ax2.step(fmgroup['starthour'], fmgroup['visby'], color = 'green', label = 'Forecast Visibility', linewidth = 5, zorder = 2)

for te in range(len(tmpo['visby'])):
  if not np.isnan(tmpo.visby[te]):
    ax2.scatter(tmpo.starthour[te], tmpo.visby[te], color = 'black', marker = '^', zorder = 3, s = 100)
    ax2.scatter(tmpo.endhour[te], tmpo.visby[te], color = 'black', marker = '^', zorder = 3, s = 100)
    x_coords = [tmpo.starthour[te], tmpo.endhour[te]]
    y_coords = [tmpo.visby[te], tmpo.visby[te]]
    ax2.plot(x_coords, y_coords, color = 'black', linewidth = 2, label = 'TEMPO', zorder = 3)
for po in range(len(pb30['visby'])):
  if not np.isnan(pb30.visby[po]):
    ax2.scatter(pb30.starthour[po], pb30.visby[po], color = 'maroon', marker = '^', zorder = 3, s = 100)
    ax2.scatter(pb30.endhour[po], pb30.visby[po], color = 'maroon', marker = '^', zorder = 3, s = 100)
    x_coords = [pb30.starthour[po], pb30.endhour[po]]
    y_coords = [pb30.visby[po], pb30.visby[po]]
    ax2.plot(x_coords, y_coords, color = 'maroon', linewidth = 2, label = 'PROB30', zorder = 3)

ax2.legend(loc = 'upper right')
ax2.set_xlabel('Hours Since TAF Began')
ax2.set_ylabel('Visibility (miles)')
ax2.set_ylim(0,7.5)
ax2.set_yticks([0,0.5,1,2,3,4,5,6,7])
ax2.set_yticklabels(['0','0.5','1','2','3','4','5','6','>6'])

if airport in Extended_Taf_List:
  ax2.set_xlim(-0.1,29.5)
  ax2.set_xticks(np.arange(0,29.5,1))
elif airport not in Extended_Taf_List:
  ax2.set_xlim(-0.1,23.5)
  ax2.set_xticks(np.arange(0,23.5,1))

plt.subplots_adjust(hspace=0.25)
plt.savefig(f'TAFVerification_{airport}_{year}{month}{day}{hour}', bbox_inches = 'tight')